# PUI2015 Final 

## DATA:

    
 <h3>  __ASTHMA DATA__:</h3> SQL query from public data in cartoDB. A helper function to query the data and more detailed instructions on the SQL queries are provided below. You should end up with the 5 NYC facilities that reported the highest number of asthma cases. 
 
 We define a helper function queryCartoDB wraps around the SQL API of CartoDB. We can pass it a public data source (in the form of 'https://{ACCOUNT}.cartodb.com/api/v2/sql', where ACCOUNT is the owner of the data) and an SQL statement to query the data in various formats including CSV, JSON, and GeoJSON. The result returned from this function is a file-like object, which you can pass to a file readers such as Pandas' read_csv() or the JSON parser. In this final exam, all SQL-based data sets have been setup to be publicly available from the account htv210, so, you don't have to use your accounts. But if you prefer, you can duplicate the data and query directly from your account. However, your data copies must also be set to public and your SQL statements must still be valid. The SQL contains three data sources:
 
1) New York City Hospital Inpatient 2013 Discharge (nyc_discharge_2013): this is a subset from SPARC data set released by the New York State Department of Health. SPARC is the Statewide Planning and Research Cooperative System Inpatient De-identified dataset contains discharge level detail on patient characteristics, diagnoses, treatments, services, and charges. This data contains basic record level detail regarding the discharge; however the data does not contain protected health information (PHI) under Health Insurance Portability and Accountability Act (HIPAA). The health information is not individually identifiable; all data elements considered identifiable have been redacted.

2) Health Facility Information (nyc_facilities): this dataset contains information on certifications for services and beds for Article 28, Article 36 and Article 40 health care facilities and programs from the Health Facilities Information System (HFIS). The information also includes the latitude and longitude of each facility.

3) Highway streets per zip code (nyc_zipcode_highway): this dataset contains the total length of highway streets per each zip code in NYC. This information was computed based on the Simplified NYS Streets for Labeling dataset and NYC zip code boundaries from the NYS GIS Clearinghouse. This data set includes the zip code boundaries and the highway length in miles within 0.25 miles of that zipcodes.

In the next cell, we're going to define the helper function. You can copy and paste this code in your notebook.


In [1]:
%matplotlib inline

import ast
import json
import urllib
import urllib2
from cStringIO import StringIO
import pandas as pd
import geopandas as gpd


SQL_SOURCE = 'https://htv210.cartodb.com/api/v2/sql'

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())

 <h3> __Health Facility Information__: </h3> SQL query from public data in cartoDB, as for the asthma data. A helper function to query the data and more detailed instructions on the SQL queries are provided below. The database also includes the zipcode of each facility.

 <h3>  __NYC ZIP CODE__ shape file :</h3> This database contains the ZIP codes geometry, the same kind of data you used in HW10 from the nyc-zip-codes shape file. The highway length per ZIP code is also given to you in a mysql database (see below); a query to the DB is shown below. 
 
   
   The next three cells are for testing if you can connect to the databases above and retrieve the first 3 records of these 3 tables. This also shows you how to create a data frame from the CartoDB's SQL API. Since the data are extracted from CartoDB, they always have the_geom and the_geom_webmercator column, but in our case, only the nyc_facilities and nyc_zipcode_highway tables have valid geospatial information. The facility_id column of the nyc_discharge_2013 table and the facilities table can be assumed to match.
   
Please also note that, in the example cells below, we're reading the nyc_discharge_2013 table as a Pandas DataFrame while the data from nyc_facilities and nyc_zipcode_highway are being put into a GeoPandas GeoDataFame.

In [2]:
dischargeQuery = '''
SELECT *
FROM nyc_discharge_2013
LIMIT 3
'''
dischargeStream = queryCartoDB(dischargeQuery, format='CSV')
discharge       = pd.read_csv(dischargeStream)
discharge.head()

cartodb_id  the_geom  the_geom_webmercator hospital_county  \
0         153       NaN                   NaN           Bronx   
1         751       NaN                   NaN           Bronx   
2         797       NaN                   NaN           Bronx   

   operating_certificate_number  facility_id age_group  zip_code___3_digits  \
0                       7000001         1164  30 to 49                  104   
1                       7000001         1164   0 to 17                  104   
2                       7000001         1164  18 to 29                  104   

  gender        race ethnicity  length_of_stay admit_day_of_week  \
0      M  Other Race   Unknown              16               THU   
1      F  Other Race   Unknown              14               FRI   
2      F  Other Race   Unknown               5               TUE   

  type_of_admission patient_disposition  discharge_year discharge_day_of_week  \
0          Elective   Home or Self Care            2013                   SAT   
1         Emergency   Home or Self Care            2013                   FRI   
2          Elective   Home or Self Care            2013                   SUN   

   ccs_diagnosis_code ccs_diagnosis_description  
0                 661   SUBSTANCE-RLTD DISORDER  
1                 659   SCHIZ/OTH PSYCH DISORDR  
2                 660   ALCOHOL-RELATD DISORDER

In [ ]:
facilityQuery = '''
SELECT *
FROM nyc_facilities
LIMIT 3

'''
facilityStream = queryCartoDB(facilityQuery, format='GeoJSON')
facilityData   = json.loads(facilityStream.read())
facilities     = gpd.GeoDataFrame.from_features(facilityData['features'])
facilities.head()

In [ ]:
zipcodeQuery = '''
SELECT *
FROM nyc_zipcode_highway
LIMIT 3
'''
zipcodeStream = queryCartoDB(zipcodeQuery, format='GeoJSON')
zipcodeData   = json.loads(zipcodeStream.read())
zipcodes      = gpd.GeoDataFrame.from_features(zipcodeData['features'])
zipcodes.head()

   
   YOU HAVE TO CHOSE AT LEAST 2 OF THE FOLLOWING 4 DATA SETS. 
 
  <h3> __HIGHWAY LENGTH__: </h3> the highway length per zip code is give to you in a mysql database as above (NYC ZIP CODE: this dataset contains the shape files for ZIP codes as well as the highway length for that zip code).
   
  <h3> __MURDER RATE__: </h3> Number of murders **by precint per year** can be found here http://www.nyc.gov/html/nypd/html/analysis_and_planning/historical_nyc_crime_data.shtml
    and the geographical location of the precinct can be found, and exported (for example as a shapefile) by searching for "Precincts" on the NYC open data portal https://data.cityofnewyork.us/ .
    Join the dataframes by precinct number (the numbers match in the 2 files) to get the precinct ZIP code, longitude and latitude, and the number of murders for each precinct (you can average over the years, or choose a specific year, in which case there is an obvious choice).
    
  <h3> __GREEN SPACES__: </h3> A file for all NYC parks can also be found at the https://data.cityofnewyork.us/ and exported (for example as a shapefile) by searching for "parks". You will need the geographical coordinates (lon and lat) and the park size (and these come in different units than the NYC zips file. A coordinate conversion is needed. There are suggestions on how to do it if you get stuck in the hints file).
    
  <h3> __311 CALLS__: </h3> 311 calls will be the metric for general environmental conditions. 311 data is a large database in nyc open data portal. Don't try and download all the data. You can choose a single year of data (the choice is obvious). Also, you will only need a few variables: geographical location and complaint type.  (I found it very useful here to plot histograms of the data).
   
   

  <h2> __DATA MUNGING AND METRCS__</h2>
   You need a metric that measures each of the relevant variables at all locations (all zipcodes since we are working at ZIP code level resolution) in NYC. 
   
   For example: to assess the importance of the access to greenspaces you need a measure of how much access to greenspaces a certain ZIP code has. You should then account for the size of nearby parks (the larger the park the more access to greenspace) and for the distance to the parks inversely (the farther the park the less access to the greenspace). Similarly for the murders you should make a metric that accounts for the rate of murders per precint, and inversely for the distance to that precinct. We make recommendations in the hints file on how to do this if these instructions are not enough. Assume the distances are from the center of the ZIP code you are considering.
   
  For the 311 calls, you need to select among all calls those that may be associated to relevant triggers, and you need to normalize those by the total number of calls per ZIP code. You can work on the ZIP code level here and count the relevant call rate per ZIP. We give suggestions in the hints file on which 311 call descriptions may be associated to asthma. You are not bound to those suggestions. 

You will need to make spatial joints to merge files by ZIP code. Remember that the columns you merge on have to have the same values, and the same types! (both integers, or both strings) 